In [1]:
from pathlib import Path
import sys
import cv2
import numpy as np
from keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
from keras.layers import Conv2D, Conv2DTranspose
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import concatenate
from keras.models import Model
from keras.optimizers import Adam

2024-05-14 15:32:54.877458: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-14 15:32:54.947442: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-14 15:32:55.979719: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Training UNET

## Importing the necessary functions

In [2]:
sys.path.append("..") # fix

In [14]:
from training.simple.conv2d_block import conv2d_block
from training.simple.read_dataset import read_dataset

In [12]:
class TrackProgress(Callback):
    def on_epoch_end(self, epoch, logs=None):
        out_progress_folder = Path("progress/best")
        if epoch == 0:
            out_progress_folder.mkdir(parents=True, exist_ok=True)
        if epoch % 10 == 0:
            _images = images_val[::16, :, :].copy()
            _masks = masks_val[::16, :, :].copy()

            n_tests = _images.shape[0]
            _pred_masks = model.predict(_images)

            _images *= 255.0
            _masks *= 255.0
            _pred_masks *= 255.0

            for i in range(n_tests):
                _img = _images[i, :, :, 0].astype(np.uint8)
                _mask = _masks[i, :, :, 0].astype(np.uint8)
                _pred_mask = _pred_masks[i, :, :, 0].astype(np.uint8)
                _out_img = np.hstack((_img, _mask, _pred_mask))
                cv2.imwrite(
                    str(
                        Path(
                            out_progress_folder,
                            f"progress_img_ex_{i:03d}_{epoch:05d}.jpg",
                        )
                    ),
                    _out_img,
                )

## Defining the path of the dataset

In [15]:
dataset_folder = Path(r"/home/higorem/DeepFlowImaging/UNET/examples/dataset_UNET")

## Defining hyperparameters

In [ ]:
window_size = 64

# number of iterations of all the training data in one cycle for training the model
EPOCHS = 5000

# number of epochs after the loss of the model get lower as possible
PATIENCE = 500

# number of samples
BATCH_SIZE = 128


## Reading the training dataset

In [ ]:
images_train, masks_train = read_dataset(
    dataset_folder, window_size=window_size, subset="Training"
)

## Reading the validation dataset

In [ ]:
images_val, masks_val = read_dataset(
    dataset_folder, window_size=window_size, subset="Validation"
)

In [ ]:
N_dataset = images_train.shape[0]
shuffle = np.arange(N_dataset)
np.random.seed(13)
np.random.shuffle(shuffle)
images_train = images_train[shuffle, :, :, :]
masks_train = masks_train[shuffle, :, :, :]

input_img = Input((window_size, window_size, 1), name="img")

c1 = conv2d_block(input_img, 8, kernel_size=5, batchnorm=True)
p1 = MaxPooling2D((4, 4))(c1)

c2 = conv2d_block(p1, 8, kernel_size=5, batchnorm=True)
p2 = MaxPooling2D((4, 4))(c2)

c3 = conv2d_block(p2, 8, kernel_size=5, batchnorm=True)

u4 = Conv2DTranspose(8, kernel_size=5, strides=(4, 4), padding="same")(c3)
u4 = concatenate([u4, c2])
c4 = conv2d_block(u4, 8, kernel_size=5, batchnorm=True)

u5 = Conv2DTranspose(8, kernel_size=5, strides=(4, 4), padding="same")(c4)
u5 = concatenate([u5, c1])
c5 = conv2d_block(u5, 8, kernel_size=5, batchnorm=True)

outputs = Conv2D(1, (1, 1), activation="sigmoid")(c5)
model = Model(inputs=[input_img], outputs=[outputs])
model.compile(optimizer=Adam(), loss="binary_crossentropy")

model.summary()

early_stopping = EarlyStopping(
    monitor="val_loss", mode="min", verbose=1, patience=PATIENCE
)

checkpoint = ModelCheckpoint(
    "UNET_best.keras", verbose=1, save_best_only=True, monitor="val_loss", mode="auto"
)

modelo.fit(
    images_train,
    masks_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(images_val, masks_val),
    callbacks=[checkpoint, early_stopping, TrackProgress()],
)